In [1]:
import numpy as np
import torch
import time
from torchvision.models import *
import pandas as pd
import os
import torchvision
from torch2trt import torch2trt

In [2]:
from torchvision.models.segmentation import *
#deeplabv3_resnet50(pretrained=False,)

In [3]:
FP32 = True
FP16 = True
INT8 = True

In [4]:
# make results
os.makedirs("results", exist_ok=True)

In [5]:
class ModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super(ModelWrapper, self).__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)['out']

In [6]:
def computeTime(model, input_size=[1, 3, 224, 224], device='cuda', FP16=False):
    inputs = torch.randn(input_size)
    if device == 'cuda':
        model = model.cuda()
        inputs = inputs.cuda()
    if FP16:
        model = model.half()
        inputs = inputs.half()

    model.eval()

    i = 0
    time_spent = []
    while i < 200:
        start_time = time.time()
        with torch.no_grad():
            _ = model(inputs)

        if device == 'cuda':
            torch.cuda.synchronize() # wait for cuda to finish (cuda is asynchronous!)
        if i != 0:
            time_spent.append(time.time() - start_time)
        i += 1
    print('Avg execution time (ms): {:.3f}'.format(np.mean(time_spent)))
    return np.mean(time_spent)

In [7]:
# resnet is enought for now
modellist = ["fcn_resnet50", "fcn_resnet101", "deeplabv3_resnet50", "deeplabv3_resnet101"]

In [ ]:
for i, model_name in enumerate(modellist):
    runtimes = []

    # define model
    print("model: {}".format(model_name))
    input_size = [1, 3, 512, 512]
    mdl = globals()[model_name]
    model = mdl().cuda().eval()
    # Run raw models
    runtimes.append(computeTime(model, input_size=input_size, device="cuda", FP16=False))

    if FP32:    
        mdl = globals()[model_name]
        model = mdl().cuda().eval()
        model_w = ModelWrapper(model)
        x = torch.zeros(input_size).cuda()

        # convert to tensorrt models
        model_trt = torch2trt(model_w, [x])

        # Run TensorRT models
        runtimes.append(computeTime(model_trt, input_size=input_size, device="cuda", FP16=False))
    if FP16:
        print("running fp16 models..")
        # Make FP16 tensorRT models
        mdl = globals()[model_name]
        model = mdl().eval().half().cuda()
        model_w = ModelWrapper(model).half()
        x = torch.zeros(input_size).half().cuda()
        # convert to tensorrt models
        model_trt = torch2trt(model_w, [x], fp16_mode=True)
        # Run TensorRT models
        runtimes.append(computeTime(model_trt, input_size=input_size, device="cuda", FP16=True))

        results.append({model_name: runtimes})
    if INT8:
        print("running int8 models..")
        # Make INT8 tensorRT models
        mdl = globals()[model_name]
        model = mdl().eval().half().cuda()
        model_w = ModelWrapper(model).half()
        x = torch.randn(input_size).half().cuda()
        # convert to tensorrt models
        model_trt = torch2trt(model_w, [x], fp16_mode=True, int8_mode=True, max_batch_size=1)

        runtimes.append(computeTime(model_trt, input_size=input_size, device="cuda", FP16=True))
        results.append({model_name: runtimes})

    if i == 0:
        df = pd.DataFrame({model_name: runtimes},
                         index = ["Raw", "FP32", "FP16", "INT8"])
    else:
        df[model_name] = runtimes

model: fcn_resnet50
Avg execution time (ms): 0.216
Avg execution time (ms): 0.174
running fp16 models..


In [ ]:
df.to_csv("results/xavier_segmentation.csv")
df